In [326]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn import set_config
set_config(display="diagram")
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve, roc_auc_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import plot_roc_curve, auc,roc_curve,roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [327]:
reviews_df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/car_reviews.csv')

In [328]:
reviews_df = reviews_df.drop(columns=['Unnamed: 0'])

In [329]:
car_data_df = pd.read_csv('/Users/mahmoud/Car_Price_Prediction/car_data_cleaned.csv')

In [330]:
car_data_df = car_data_df.drop(columns=['Unnamed: 0'])

In [337]:
car_data_df

,Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Year,Make,Model,Type
0,2015 Ford C-Max Hybrid SEL,88380,Capitol Kia,3.8,(180 reviews),10998.0,2015,Ford C,SEL,Hybrid SEL
1,2016 Kia Sorento LX,100837,Napleton Mazda of Naperville,3.9,(115 reviews),12577.0,2016,Kia Sorento LX,LX,LX
2,2016 Mitsubishi Lancer ES,133750,AVA Auto Sales,0.0,(8 reviews),8495.0,2016,Mitsubishi Lancer ES,ES,ES
3,2011 Ford Escape XLT,110862,94 Nissan Of South Holland,4.7,"(2,682 reviews)",7869.0,2011,Ford Escape XLT,XLT,XLT
4,2017 Dodge Journey SXT,58595,Lithia Chrysler Jeep Dodge Ram of Wasilla,4.6,(543 reviews),14888.0,2017,Dodge Journey SXT,SXT,SXT
...,...,...,...,...,...,...,...,...,...,...
10033,2014 Chevrolet Silverado 1500 LT,158598,City Auto Sales Corp,0.0,(1 review),13995.0,2014,Chevrolet Silverado,LT,1500 LT
10034,2019 Hyundai Sonata SE,113581,Auto Boutique,4.8,"(1,074 reviews)",13500.0,2019,Hyundai Sonata SE,SE,SE
10035,2013 Dodge Journey SXT,197496,Ryan Auto Mall Chrysler Dodge Jeep Ram Of Mont...,0.0,(29 reviews),7425.0,2013,Dodge Journey SXT,SXT,SXT
10036,2007 Lexus ES 350,85384,"Mountain Motors, Inc",0.0,(7 reviews),11995.0,2007,Lexus ES,NaN,350


In [332]:
# Having the mileage column in float rather than string
car_data_df['Mileage'] = car_data_df['Mileage'].str.replace(' mi.', '').str.replace(',', '')
car_data_df['Mileage'] = pd.to_numeric(car_data_df['Mileage'], errors='coerce')
car_data_df['Mileage']

0         88380
1        100837
2        133750
3        110862
4         58595
          ...  
10033    158598
10034    113581
10035    197496
10036     85384
10037    133800
Name: Mileage, Length: 10038, dtype: int64

In [322]:
reviews_df[['Year', 'Make', 'Model', 'Type']] = reviews_df['Car Name'].str.extract(r'(\d{4})\s+([^ ]+)\s+([^ ]+)\s+(.*)')
reviews_df

,Car Name,Rating,Reviews,Year,Make,Model,Type
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra,PRO-4X
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra,SE
2,2017 Subaru Impreza 2.0i Sport,5.0,"['November 2, 2022\nBy Sophie from Horse Cave...",2017,Subaru,Impreza,2.0i Sport
3,2014 Cadillac ATS 2.5L Luxury,4.5,"['May 23, 2023\nBy Helene from Hallandale Beac...",2014,Cadillac,ATS,2.5L Luxury
4,2013 Mercedes-Benz E-Class E 350,4.8,"['March 13, 2021\nBy My personal review from W...",2013,Mercedes-Benz,E-Class,E 350
...,...,...,...,...,...,...,...
9889,2013 Chevrolet Tahoe LT,1.3,"['May 18, 2021\nBy Mel.R from Boston MA\nOwns ...",2013,Chevrolet,Tahoe,LT
9890,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535,xDrive
9891,2014 Chrysler 200 Limited,4.2,"['May 16, 2023\nBy Coach Joe from Clio MI.\nOw...",2014,Chrysler,200,Limited
9892,2011 Honda CR-V LX,4.5,"['May 9, 2021\nBy Sheila C from Phoenix AZ\nO...",2011,Honda,CR-V,LX


In [333]:
reviews_df.isna().sum()

Car Name    0
Rating      0
Reviews     0
dtype: int64

In [334]:
# Clarify the column Names
reviews_df = reviews_df.rename(columns={"Rating": "Car Rating"})
car_data_df = car_data_df.rename(columns={"Rating": "Dealer Rating"})

In [335]:
merge1 = car_data_df.merge(reviews_df, how = 'left', left_on =['Name', 'Year'] ,right_on = ['Car Name', 'Year'])

KeyError: 'Year'

In [276]:
merge = reviews_df.merge(car_data_df, how = 'left', left_on =['Car Name', 'Year'] ,right_on = ['Name', 'Year'])
merge

,Car Name,Car Rating,Reviews,Year,Make_x,Model_x,Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Make_y,Model_y,Type,Dealer Rating
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra PRO-4X,2013 Nissan Xterra PRO-4X,163000.0,Priced Right Auto,0.0,(19 reviews),9500.0,Nissan Xterra PRO,4X,PRO-4X,0.0
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,95747.0,Blasius Pre-Owned Auto,4.4,(75 reviews),8994.0,Hyundai Elantra SE,SE,SE,4.4
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,159177.0,Metro Motors LLC,0.0,(1 review),8900.0,Hyundai Elantra SE,SE,SE,0.0
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,79055.0,Oakbrook Toyota in Westmont,4.5,"(2,527 reviews)",10304.0,Hyundai Elantra SE,SE,SE,4.5
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,102894.0,Sawari Automotive Group,4.7,(31 reviews),9599.0,Hyundai Elantra SE,SE,SE,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,146541.0,Best Choice Auto Sales,0.0,(37 reviews),9990.0,BMW,xDrive,xDrive,0.0
26523,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,103001.0,MC Auto Mart LLC,0.0,0,13999.0,BMW,xDrive,xDrive,0.0
26524,2014 Chrysler 200 Limited,4.2,"['May 16, 2023\nBy Coach Joe from Clio MI.\nOw...",2014,Chrysler,200 Limited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26525,2011 Honda CR-V LX,4.5,"['May 9, 2021\nBy Sheila C from Phoenix AZ\nO...",2011,Honda,CR-V LX,2011 Honda CR-V LX,183232.0,BMW of Monterey,4.5,(959 reviews),7991.0,Honda CR,LX,LX,4.5


# Work with merge dataset

In [270]:
merge.isna().sum()

Car Name            0
Car Rating         63
Reviews             0
Year                0
Make_x              0
Model_x             0
Name             2581
Mileage          2581
Dealer Name      2581
Dealer Rating    2581
Review Count     2581
Price            2581
Make_y           2581
Model_y          2689
Type             2584
Dealer Rating    2581
dtype: int64

In [294]:
clean_merge.isna().sum()

Car Name          0
Car Rating       19
Reviews           0
Year              0
Make_x            0
Model_x           0
Name_x            0
Mileage           0
Dealer Name       0
Dealer Rating     0
Review Count      0
Price             0
Make_y            0
Model_y           0
Type              2
Dealer Rating     0
Name_y            0
dtype: int64

In [ ]:
# merge = merge.loc[:, ~merge.columns.duplicated()]

# Merge the DataFrame based on 'Car Name' and 'Name' columns, keeping only matching rows
clean_merge = pd.merge(merge, merge[['Name']], left_on='Car Name', right_on='Name', how='inner')

# Now, clean_merge contains only the rows where 'Car Name' matches 'Name'


In [295]:
# Check for duplicates 
duplicates = clean_merge.duplicated()

# Count the number of duplicate rows
num_duplicates = duplicates.sum()

# Display the number of duplicate rows and the duplicate rows themselves
print(clean_merge[duplicates])


Empty DataFrame
Columns: [Car Name, Car Rating, Reviews, Year, Make_x, Model_x, Name_x, Mileage, Dealer Name, Dealer Rating, Review Count, Price, Make_y, Model_y, Type, Dealer Rating, Name_y]
Index: []


In [286]:
# Drop the duplicated rows
clean_merge = clean_merge.drop_duplicates()

# Check the number of rows after dropping duplicates
num_rows_after_dropping_duplicates = len(clean_merge)

# Display the updated DataFrame and the number of rows after dropping duplicates
print(clean_merge)
print("Number of rows after dropping duplicates:", num_rows_after_dropping_duplicates)


                                         Car Name  Car Rating  \
0                       2013 Nissan Xterra PRO-4X         4.5   
9                         2014 Hyundai Elantra SE         4.5   
69                        2014 Hyundai Elantra SE         4.5   
129                       2014 Hyundai Elantra SE         4.5   
189                       2014 Hyundai Elantra SE         4.5   
...                                           ...         ...   
1365093  2005 Dodge Ram 1500 SLT/Laramie Quad Cab         5.0   
1365095  2005 Dodge Ram 1500 SLT/Laramie Quad Cab         5.0   
1365097                 2014 Audi A4 Premium Plus         4.4   
1365098                       2011 BMW 535 xDrive         4.5   
1365100                       2011 BMW 535 xDrive         4.5   

                                                   Reviews  Year   Make_x  \
0        ['March 15, 2021\nBy Tony from Detroit\nOwns t...  2013   Nissan   
9        ['May  7, 2022\nBy Talessa from Tulsa, ok\nOwn...  2014 

In [287]:
clean_merge

,Car Name,Car Rating,Reviews,Year,Make_x,Model_x,Name_x,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Make_y,Model_y,Type,Dealer Rating,Name_y
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra PRO-4X,2013 Nissan Xterra PRO-4X,163000.0,Priced Right Auto,0.0,(19 reviews),9500.0,Nissan Xterra PRO,4X,PRO-4X,0.0,2013 Nissan Xterra PRO-4X
9,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,95747.0,Blasius Pre-Owned Auto,4.4,(75 reviews),8994.0,Hyundai Elantra SE,SE,SE,4.4,2014 Hyundai Elantra SE
69,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,159177.0,Metro Motors LLC,0.0,(1 review),8900.0,Hyundai Elantra SE,SE,SE,0.0,2014 Hyundai Elantra SE
129,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,79055.0,Oakbrook Toyota in Westmont,4.5,"(2,527 reviews)",10304.0,Hyundai Elantra SE,SE,SE,4.5,2014 Hyundai Elantra SE
189,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,102894.0,Sawari Automotive Group,4.7,(31 reviews),9599.0,Hyundai Elantra SE,SE,SE,4.7,2014 Hyundai Elantra SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365093,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,171430.0,Broadway Auto Mall,0.0,(16 reviews),5980.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
1365095,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,153560.0,Trucks Plus Idaho,0.0,(2 reviews),9988.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
1365097,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,Audi,A4 Premium Plus,2014 Audi A4 Premium Plus,138173.0,Your Choice Auto Sales of Elgin,4.4,(57 reviews),12999.0,Audi A,Plus,Premium Plus,4.4,2014 Audi A4 Premium Plus
1365098,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,146541.0,Best Choice Auto Sales,0.0,(37 reviews),9990.0,BMW,xDrive,xDrive,0.0,2011 BMW 535 xDrive


- From 27k rows in 'merge' to now 14k rows

In [289]:
clean_merge.isna().sum()

Car Name          0
Car Rating       19
Reviews           0
Year              0
Make_x            0
Model_x           0
Name_x            0
Mileage           0
Dealer Name       0
Dealer Rating     0
Review Count      0
Price             0
Make_y            0
Model_y          77
Type              2
Dealer Rating     0
Name_y            0
dtype: int64

In [290]:
car_rating_mean = clean_merge['Car Rating'].mean()
car_rating_mean

4.269581109730244

- either we fill it with the mean or zero's

- for the Model_y Nans, we will fill it in using Model_x

In [292]:
clean_merge.loc['Model_y'] = clean_merge.loc['Model_y'].fillna(clean_merge.loc['Model_x'])

KeyError: 'Model_y'

In [297]:
clean_merge.isna().sum()

Car Name          0
Car Rating       19
Reviews           0
Year              0
Make_x            0
Model_x           0
Name_x            0
Mileage           0
Dealer Name       0
Dealer Rating     0
Review Count      0
Price             0
Make_y            0
Model_y           0
Type              2
Dealer Rating     0
Name_y            0
dtype: int64

In [299]:
# Drop NaN in Type
clean_merge = clean_merge.dropna(subset=['Type'])

0


In [300]:
# we will fill the nans in Car rating by the mean
clean_merge['Car Rating'] = clean_merge['Car Rating'].fillna(car_rating_mean)

<ipython-input-300-a82d7fbc5b9a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_merge['Car Rating'] = clean_merge['Car Rating'].fillna(car_rating_mean)


In [301]:
clean_merge.isna().sum()

Car Name         0
Car Rating       0
Reviews          0
Year             0
Make_x           0
Model_x          0
Name_x           0
Mileage          0
Dealer Name      0
Dealer Rating    0
Review Count     0
Price            0
Make_y           0
Model_y          0
Type             0
Dealer Rating    0
Name_y           0
dtype: int64

# No more Nulls

In [303]:
merge

,Car Name,Car Rating,Reviews,Year,Make_x,Model_x,Name,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Make_y,Model_y,Type
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra PRO-4X,2013 Nissan Xterra PRO-4X,163000.0,Priced Right Auto,0.0,(19 reviews),9500.0,Nissan Xterra PRO,4X,PRO-4X
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,95747.0,Blasius Pre-Owned Auto,4.4,(75 reviews),8994.0,Hyundai Elantra SE,SE,SE
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,159177.0,Metro Motors LLC,0.0,(1 review),8900.0,Hyundai Elantra SE,SE,SE
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,79055.0,Oakbrook Toyota in Westmont,4.5,"(2,527 reviews)",10304.0,Hyundai Elantra SE,SE,SE
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,102894.0,Sawari Automotive Group,4.7,(31 reviews),9599.0,Hyundai Elantra SE,SE,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,146541.0,Best Choice Auto Sales,0.0,(37 reviews),9990.0,BMW,xDrive,xDrive
26523,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,103001.0,MC Auto Mart LLC,0.0,0,13999.0,BMW,xDrive,xDrive
26524,2014 Chrysler 200 Limited,4.2,"['May 16, 2023\nBy Coach Joe from Clio MI.\nOw...",2014,Chrysler,200 Limited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26525,2011 Honda CR-V LX,4.5,"['May 9, 2021\nBy Sheila C from Phoenix AZ\nO...",2011,Honda,CR-V LX,2011 Honda CR-V LX,183232.0,BMW of Monterey,4.5,(959 reviews),7991.0,Honda CR,LX,LX


In [308]:
clean_merge['Review Count'] = clean_merge['Review Count'].astype(str).str.extract(r'(\d+)').astype(float)
clean_merge['Price'] = clean_merge['Price'].astype(str).str.replace('[^0-9.]', '', regex=True).astype(float)

clean_merge.reset_index(drop=True, inplace=True)

Car Name         0
Car Rating       0
Reviews          0
Year             0
Make_x           0
Model_x          0
Name_x           0
Mileage          0
Dealer Name      0
Dealer Rating    0
Review Count     0
Price            0
Make_y           0
Model_y          0
Type             0
Dealer Rating    0
Name_y           0
dtype: int64
                                       Car Name  Car Rating  \
0                     2013 Nissan Xterra PRO-4X         4.5   
1                       2014 Hyundai Elantra SE         4.5   
2                       2014 Hyundai Elantra SE         4.5   
3                       2014 Hyundai Elantra SE         4.5   
4                       2014 Hyundai Elantra SE         4.5   
...                                         ...         ...   
13691  2005 Dodge Ram 1500 SLT/Laramie Quad Cab         5.0   
13692  2005 Dodge Ram 1500 SLT/Laramie Quad Cab         5.0   
13693                 2014 Audi A4 Premium Plus         4.4   
13694                       2011 B

<ipython-input-308-ab935da9e13c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_merge['Review Count'] = clean_merge['Review Count'].astype(str).str.extract(r'(\d+)').astype(float)
<ipython-input-308-ab935da9e13c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_merge['Price'] = clean_merge['Price'].astype(str).str.replace('[^0-9.]', '', regex=True).astype(float)


In [309]:
clean_merge

,Car Name,Car Rating,Reviews,Year,Make_x,Model_x,Name_x,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Make_y,Model_y,Type,Dealer Rating,Name_y
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra PRO-4X,2013 Nissan Xterra PRO-4X,163000.0,Priced Right Auto,0.0,19.0,9500.0,Nissan Xterra PRO,4X,PRO-4X,0.0,2013 Nissan Xterra PRO-4X
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,95747.0,Blasius Pre-Owned Auto,4.4,75.0,8994.0,Hyundai Elantra SE,SE,SE,4.4,2014 Hyundai Elantra SE
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,159177.0,Metro Motors LLC,0.0,1.0,8900.0,Hyundai Elantra SE,SE,SE,0.0,2014 Hyundai Elantra SE
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,79055.0,Oakbrook Toyota in Westmont,4.5,2527.0,10304.0,Hyundai Elantra SE,SE,SE,4.5,2014 Hyundai Elantra SE
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,102894.0,Sawari Automotive Group,4.7,31.0,9599.0,Hyundai Elantra SE,SE,SE,4.7,2014 Hyundai Elantra SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,171430.0,Broadway Auto Mall,0.0,16.0,5980.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,153560.0,Trucks Plus Idaho,0.0,2.0,9988.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,Audi,A4 Premium Plus,2014 Audi A4 Premium Plus,138173.0,Your Choice Auto Sales of Elgin,4.4,57.0,12999.0,Audi A,Plus,Premium Plus,4.4,2014 Audi A4 Premium Plus
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,146541.0,Best Choice Auto Sales,0.0,37.0,9990.0,BMW,xDrive,xDrive,0.0,2011 BMW 535 xDrive


get rid of Make_x, Model_x, Make_x, Make_y, Type,and Name_X and Name_y(since both are now the exact same thing as Car_Name)

In [310]:
deep_clean = clean_merge.drop(['Make_x', 'Model_x', 'Name_x', 'Make_y', 'Model_y', 'Type', 'Name_y'], axis=1)
deep_clean

,Car Name,Car Rating,Reviews,Year,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Dealer Rating
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,163000.0,Priced Right Auto,0.0,19.0,9500.0,0.0
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,95747.0,Blasius Pre-Owned Auto,4.4,75.0,8994.0,4.4
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,159177.0,Metro Motors LLC,0.0,1.0,8900.0,0.0
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,79055.0,Oakbrook Toyota in Westmont,4.5,2527.0,10304.0,4.5
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,102894.0,Sawari Automotive Group,4.7,31.0,9599.0,4.7
...,...,...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,171430.0,Broadway Auto Mall,0.0,16.0,5980.0,0.0
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,153560.0,Trucks Plus Idaho,0.0,2.0,9988.0,0.0
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,138173.0,Your Choice Auto Sales of Elgin,4.4,57.0,12999.0,4.4
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,146541.0,Best Choice Auto Sales,0.0,37.0,9990.0,0.0


In [311]:
# Drop one of the 'Dealer Rating' columns from deep_clean
deep_clean = deep_clean.drop('Dealer Rating', axis=1)

In [312]:
deep_clean

,Car Name,Car Rating,Reviews,Year,Mileage,Dealer Name,Review Count,Price
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,163000.0,Priced Right Auto,19.0,9500.0
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,95747.0,Blasius Pre-Owned Auto,75.0,8994.0
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,159177.0,Metro Motors LLC,1.0,8900.0
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,79055.0,Oakbrook Toyota in Westmont,2527.0,10304.0
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,102894.0,Sawari Automotive Group,31.0,9599.0
...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,171430.0,Broadway Auto Mall,16.0,5980.0
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,153560.0,Trucks Plus Idaho,2.0,9988.0
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,138173.0,Your Choice Auto Sales of Elgin,57.0,12999.0
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,146541.0,Best Choice Auto Sales,37.0,9990.0


In [339]:
clean_merge

,Car Name,Car Rating,Reviews,Year,Make_x,Model_x,Name_x,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Make_y,Model_y,Type,Dealer Rating,Name_y
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,Nissan,Xterra PRO-4X,2013 Nissan Xterra PRO-4X,163000.0,Priced Right Auto,0.0,19.0,9500.0,Nissan Xterra PRO,4X,PRO-4X,0.0,2013 Nissan Xterra PRO-4X
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,95747.0,Blasius Pre-Owned Auto,4.4,75.0,8994.0,Hyundai Elantra SE,SE,SE,4.4,2014 Hyundai Elantra SE
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,159177.0,Metro Motors LLC,0.0,1.0,8900.0,Hyundai Elantra SE,SE,SE,0.0,2014 Hyundai Elantra SE
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,79055.0,Oakbrook Toyota in Westmont,4.5,2527.0,10304.0,Hyundai Elantra SE,SE,SE,4.5,2014 Hyundai Elantra SE
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,Hyundai,Elantra SE,2014 Hyundai Elantra SE,102894.0,Sawari Automotive Group,4.7,31.0,9599.0,Hyundai Elantra SE,SE,SE,4.7,2014 Hyundai Elantra SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,171430.0,Broadway Auto Mall,0.0,16.0,5980.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,Dodge,Ram 1500 SLT/Laramie Quad Cab,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,153560.0,Trucks Plus Idaho,0.0,2.0,9988.0,Dodge Ram,Cab,1500 SLT/Laramie Quad Cab,0.0,2005 Dodge Ram 1500 SLT/Laramie Quad Cab
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,Audi,A4 Premium Plus,2014 Audi A4 Premium Plus,138173.0,Your Choice Auto Sales of Elgin,4.4,57.0,12999.0,Audi A,Plus,Premium Plus,4.4,2014 Audi A4 Premium Plus
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,BMW,535 xDrive,2011 BMW 535 xDrive,146541.0,Best Choice Auto Sales,0.0,37.0,9990.0,BMW,xDrive,xDrive,0.0,2011 BMW 535 xDrive


In [340]:
deep_mege = clean_merge.drop(['Make_x', 'Model_x', 'Name_x', 'Make_y', 'Model_y', 'Type', 'Name_y'], axis=1)
deep_mege

,Car Name,Car Rating,Reviews,Year,Mileage,Dealer Name,Dealer Rating,Review Count,Price,Dealer Rating
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,163000.0,Priced Right Auto,0.0,19.0,9500.0,0.0
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,95747.0,Blasius Pre-Owned Auto,4.4,75.0,8994.0,4.4
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,159177.0,Metro Motors LLC,0.0,1.0,8900.0,0.0
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,79055.0,Oakbrook Toyota in Westmont,4.5,2527.0,10304.0,4.5
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,102894.0,Sawari Automotive Group,4.7,31.0,9599.0,4.7
...,...,...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,171430.0,Broadway Auto Mall,0.0,16.0,5980.0,0.0
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5.0,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,153560.0,Trucks Plus Idaho,0.0,2.0,9988.0,0.0
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,138173.0,Your Choice Auto Sales of Elgin,4.4,57.0,12999.0,4.4
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,146541.0,Best Choice Auto Sales,0.0,37.0,9990.0,0.0


In [344]:
# deep_merge.rename(columns={'Dealer Rating': 'Dealer Rating_x'}, inplace=True)
# deep_mege

df2 = deep_mege.T.drop_duplicates().T


In [347]:
final_data = df2
final_data

,Car Name,Car Rating,Reviews,Year,Mileage,Dealer Name,Dealer Rating,Review Count,Price
0,2013 Nissan Xterra PRO-4X,4.5,"['March 15, 2021\nBy Tony from Detroit\nOwns t...",2013,163000,Priced Right Auto,0,19,9500
1,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,95747,Blasius Pre-Owned Auto,4.4,75,8994
2,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,159177,Metro Motors LLC,0,1,8900
3,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,79055,Oakbrook Toyota in Westmont,4.5,2527,10304
4,2014 Hyundai Elantra SE,4.5,"['May 7, 2022\nBy Talessa from Tulsa, ok\nOwn...",2014,102894,Sawari Automotive Group,4.7,31,9599
...,...,...,...,...,...,...,...,...,...
13691,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,171430,Broadway Auto Mall,0,16,5980
13692,2005 Dodge Ram 1500 SLT/Laramie Quad Cab,5,"['May 8, 2023\nBy Michael from Nashville \nO...",2005,153560,Trucks Plus Idaho,0,2,9988
13693,2014 Audi A4 Premium Plus,4.4,"['September 7, 2022\nBy SInce59 from Napa, Ca...",2014,138173,Your Choice Auto Sales of Elgin,4.4,57,12999
13694,2011 BMW 535 xDrive,4.5,"['June 2, 2022\nBy Manuel from Liberal Ks\nOw...",2011,146541,Best Choice Auto Sales,0,37,9990


In [348]:
final_data.to_csv('/Users/mahmoud/Car_Price_Prediction/merged_official_data.csv')